In [8]:
import numpy as np
import pandas as pd
import feather

In [2]:
n = 600
s1 = [0,0,0]
s2 = [1,0,0]
s3 = [0,1,0]
s4 = [0,0,1]
s5 = [1,1,0]
s6 = [0,1,1]
s7 = [1,1,1]

In [3]:
config1 = np.matrix( [s7 for i in range(n)] + [s1 for i in range(n)] )
# extreme situation 1, [1,1,1] and [0,0,0], first gene is causal
# 1 indicates case, 0 indicates ctrl
sample1_ext = np.matrix( [[1] for i in range(n)] + [[0] for i in range(n)] )

In [4]:
d_const = 0.5
config2 = np.matrix( [s5 for i in range(n)] + [s6 for i in range(n)] )
# extreme situation 2, [1,1,0] and [0,1,1], first gene is causal
sample2_ext = np.matrix( [[1] for i in range(n)] + [[0] for i in range(n)] )
# mild situation 1, [1,1,0] and [0,1,1]
sample2_mild = [np.random.normal(0,1)+d_const for i in range(n)] + [np.random.normal(0,1) for i in range(n)]
sample2_mild = np.matrix( [[1] if i>np.median(sample2_mild) else [0] for i in sample2_mild] )

In [10]:
s1 = np.concatenate((sample1_ext, config1), axis=1)
s2_ext = np.concatenate((sample2_ext, config2), axis=1)
s2_mild = np.concatenate((sample2_mild, config2), axis=1)
print (s2_mild.shape[1])

4


In [30]:
df = pd.DataFrame(s2_mild, columns = ['phenotype'] + np.array(range(s2_mild.shape[1]-1), dtype = 'str').tolist())
for col in df.columns:
    df[col] = df[col].astype(np.uint8)
feather.write_dataframe(df, "data/s2_mild.feather")